## Description
In this notebook, we analyze generate the query and analyze the results of contaminant screening blast.   
We remove target proteins for which we didn't detect a fungal protein on the same contig.  
A fungal protein is defined as a protein for which the first non-self blast hit is fungal.   

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os
from Bio import SeqIO
from collections import Counter
from ete3 import NCBITaxa
from skbio.alignment import StripedSmithWaterman, local_pairwise_align_protein, local_pairwise_align_ssw
from Bio.Align import substitution_matrices
from Bio import pairwise2
from path import Path
ncbi = NCBITaxa()

## Data

In [ ]:
fungi_taxid = 4751

In [ ]:
acc2scaffold_file = 'acc2scaffold.tsv'
acc2scaffold = {}
with open(acc2scaffold_file) as h:
    for l in h:
        l = l.strip().split('\t')
        assert l[0] not in acc2scaffold
        acc2scaffold[l[0]] = l[1]
len(acc2scaffold)  # note: with piromyces sp E2

In [ ]:
target_accessions = open('basal_accessions.txt')
target_accessions = set(l.strip() for l in target_accessions if l.strip())
len(target_accessions)

In [ ]:
target_sequences_file = 'all_proteins.fa'

In [ ]:
taxid_table = open('final_homolog_taxid_table')
taxid_table = {l.strip().split()[0]: l.strip().split()[1] for l in taxid_table if l.strip()}

In [ ]:
acc2filename_file = 'acc2filename.txt'
acc2filename = [l.strip().replace('.fa', '').split('\t') for l in open(acc2filename_file) if l.strip()]
acc2filename = {l[0]: l[1] for l in acc2filename}

Input variables for section 1:

In [ ]:
contaminant_blast_dir = Path('contaminant_filtering_blast/')
input_cluster_directory = Path('first_round_clusters/')

Input variables for section 2:

In [ ]:
contaminant_blast_file = contaminant_blast_dir / 'blast_results'

Output variable:

In [ ]:
filtered_cluster_directory = Path('first_round_filtered_clusters')

## Generating contaminant filtering blast query

Here, for each remaining targer sequence, we'll take 10 random proteins from its conting.  
They will be used as a query for BLASTp.   

Step 1: Select target-fungal proteins from clusters and proteins from the same contigs  

In [ ]:
# WARNING: This will erase contents of contaminant_filtering_blast!
# However, the processing in the next stage will still be valid, only inferred from 
# a different set of sequences.  
fasta_dir = input_cluster_directory
sequences_to_scan_dir = contaminant_blast_dir / 'sequences'
print('Processing sequences in', fasta_dir)
print('Output directory:', sequences_to_scan_dir)

cluster_fasta_files = os.listdir(fasta_dir)
print('Loaded %i clusters' % len(cluster_fasta_files))
try:  # creating sub-directory
    os.mkdir(sequences_to_scan_dir)
except FileExistsError:
    dir_contents = os.listdir(sequences_to_scan_dir)
    for f in dir_contents:
        os.remove(sequences_to_scan_dir + '/' + f)


# Select target accessions from clusters with potential HGTs
target_seqids_in_clusters = []
for fstfl in cluster_fasta_files:
    sequences = list(SeqIO.parse(fasta_dir / fstfl, 'fasta'))
    tseqids = [s.id for s in sequences if s.id in target_accessions]
    target_seqids_in_clusters.extend(tseqids)
print('Selected %i target sequences' % len(target_seqids_in_clusters))
# Select accessions of other proteins from the same scaffolds 
selected_scaffolds = list(set([acc2scaffold[acc] for acc in target_seqids_in_clusters]))
print('Selected %i scaffolds' % len(selected_scaffolds))
scaffold_taken_counts = Counter()  # nbs of proteins taken per scaffolds
seqids_to_scan = set()  # accessions from neighbouring proteins in scaffolds, to be blasted against NR
for acc in acc2scaffold:
    if acc not in target_seqids_in_clusters:  # we don't want to scan potential HGTs
        scf = acc2scaffold[acc]
        if scf in selected_scaffolds:
            if scaffold_taken_counts[scf] < 10:
                scaffold_taken_counts[scf] += 1
                seqids_to_scan.add(acc)
assert all(acc in target_accessions for acc in seqids_to_scan)
print('Selected %i sequences to scan' % len(seqids_to_scan))
# Save the query sequences:
sequences_to_write = []
all_target_sequences = SeqIO.parse(target_sequences_file, 'fasta')
for s in all_target_sequences:
    if s.id in seqids_to_scan:
        sequences_to_write.append(s)
saved_seq_nb = 0
for i in range(0, len(sequences_to_write), 100):
    with open(sequences_to_scan_dir + '/chunk%i.fa' % i, 'w') as h:
        saved_seq_nb += SeqIO.write(sequences_to_write[i:(i+100)], h, 'fasta')
print('Saved', saved_seq_nb, 'contig neighbour sequences to scan')
# with open(fasta_dir + '_to_scan.fa', 'w') as h:
#     SeqIO.write(sequences_to_write, h, 'fasta')

*Now it's time to run BLASTp manually.* 

## Detecting contaminants

In this part, we analyze the resuts of blastp vs NR of proteins selected in the previous stage.   
Do not run the previous section again! Just use this one.  

Parse the results of contaminant screening blast:

In [ ]:
acc2homolog_taxids = {}  # list of taxids of homologs sorted by increasing e-value
blast_results = open(contaminant_blast_file)
for l in blast_results:
    l = l.strip().split('\t')
    if l[-1] == "N/A":  # Note: l[-1] == taxid; we're only interested in checking if the closest non-self hit is fungal
        continue
    try:
        acc2homolog_taxids[l[0]].append((int(l[-1]), float(l[-2])))  # taxid and evalue
    except KeyError:
        acc2homolog_taxids[l[0]] = [(int(l[-1]), float(l[-2]))]
print('Parsed blast results for', len(acc2homolog_taxids), 'query accessions')
# For each target protein, sort its homologs according to the e-value
for k in acc2homolog_taxids:
    acc2homolog_taxids[k] = sorted(acc2homolog_taxids[k], key = lambda x: x[1])

For each contig, obtain the ancestry score (= the number of proteins which have a fungal first hit, except host)

In [ ]:
ancestry_scores = Counter()  
i = 0
for acc in acc2homolog_taxids:
    tx_evalue_list = acc2homolog_taxids[acc]
    query_taxid = tx_evalue_list[0][0] # assuming self is the closest hit
    try:
        first_hit_tx = next((tx for tx, ev in tx_evalue_list if tx != query_taxid))
    except StopIteration:
        continue
    lineage = ncbi.get_lineage(first_hit_tx)
    if fungi_taxid in lineage:
        scaffold = acc2scaffold[acc]
        ancestry_scores[scaffold] += 1
print('Obtained scores for %i scaffolds' % len(ancestry_scores))
print('%i scaffolds have a score greater than 1' % sum(ancestry_scores[scf] > 1 for scf in ancestry_scores))

Detect contaminants (= target proteins on scaffolds with no detected fungal protein)

In [ ]:
contaminant_seq_file = contaminant_blast_dir / 'identified_contaminants.fa'
print('Input dir:', raw_cluster_dir)
print('Output file:', contaminant_seq_file)
cluster_fasta_files = os.listdir(input_cluster_directory)
print('Loaded %i cluster fastas' % len(cluster_fasta_files))

target_seqids_in_clusters = []
for fstfl in cluster_fasta_files:
    sequences = list(SeqIO.parse(input_cluster_directory  / fstfl, 'fasta'))
    tseqids = [s.id for s in sequences if s.id in target_accessions]
    target_seqids_in_clusters.extend(tseqids)
print('Selected %i target sequences' % len(target_seqids_in_clusters))

retained_sequence_nb_per_score = np.zeros(6, dtype='int')
for acc in target_seqids_in_clusters:
    scaffold = acc2scaffold[acc]
    score = ancestry_scores[scaffold]
    retained_sequence_nb_per_score[:(score+1)] += 1
print('Score threshold vs number of retained sequences:')
for i, snb in enumerate(retained_sequence_nb_per_score):
    print(i, snb)
    
score_threshold = 1
contaminant_seqids = set()
for fstfl in cluster_fasta_files:
    sequences = list(SeqIO.parse(input_cluster_directory / fstfl, 'fasta'))
    for s in sequences:
        if s.id in target_accessions:
            scaffold = acc2scaffold[s.id]
            score = ancestry_scores[scaffold]
            if score < score_threshold:
                contaminant_seqids.add(s.id)

print('Detected %i potential contaminants' % len(contaminant_seqids))

# target_sequences_file = 'all_proteins.fa'
# contaminants = []
# all_target_sequences = SeqIO.parse(target_sequences_file, 'fasta')
# for seq in all_target_sequences:
#     if seq.id in contaminant_seqids:
#         contaminants.append(seq)
# assert len(contaminants) == len(contaminant_seqids)
# with open(contaminant_seq_file, 'w') as h:
#     SeqIO.write(contaminants, h, 'fasta')

Save the identified contaminants:   

In [ ]:
with open('Results/contaminants_filter2.tsv', 'w') as h:
    for sqid in contaminant_seqids:
        h.write(sqid + '\t' + acc2filename[sqid] + '\n')

Remove contaminants and save filtered clusters

In [ ]:
print('Input dir:', raw_cluster_dir)
print('Output dir:', filtered_cluster_directory)

try:
    os.mkdir(filtered_cluster_directory)
except FileExistsError:
    dir_contents = os.listdir(filtered_cluster_directory)
    for f in dir_contents:
        os.remove(filtered_cluster_directory + '/' + f)

print('Loaded %i cluster fastas' % len(cluster_fasta_files))
removed_contaminants = []  # for testing purposes
no_fungi_clusters = 0
predominantly_fungal_clusters = 0
large_fungal_clusters = 0
saved_clusters = 0
saved_sequences = 0
saved_target_sequences = 0
all_sequences_in_clusters = []
for fstfl in cluster_fasta_files:
    sequences = list(SeqIO.parse(input_cluster_directory, 'fasta'))
    seqids_to_save = [s.id for s in sequences if s.id not in contaminant_seqids]
    removed_contaminants += [s.id for s in sequences if s.id in contaminant_seqids]
    # Check if contains any target sequences:
    if not any(seqid in target_accessions for seqid in seqids_to_save):
        no_fungi_clusters += 1
        continue
    # Check if ORFan
    seqtx = set(int(taxid_table[sqid]) for sqid in seqids_to_save)
    assert all(0 < tx for tx in seqtx)
    seqlin = ncbi.get_lineage_translator(seqtx)
    fungaltx = {tx for tx in seqlin if fungi_taxid in seqlin[tx]}
    # fungal_sequences = [seqid for seqid in seqids_to_save if taxid_table[seqid] in fungaltx]
    nbs_of_species = len(set(seqtx))
    nbs_of_fungi = len(fungaltx)
    if nbs_of_fungi >= 0.6*nbs_of_species :
        predominantly_fungal_clusters += 1
        continue
#     elif len(fungal_sequences) > 20:
#         large_fungal_clusters += 1
#         continue
        
    sequences_to_save = [s for s in sequences if s.id in seqids_to_save]
    all_sequences_in_clusters.extend(sequences_to_save)
    assert sequences_to_save
    with open(filtered_cluster_directory / fstfl, 'w') as h:
        SeqIO.write(sequences_to_save, h, 'fasta')
    saved_sequences += len(sequences_to_save)
    saved_clusters += 1
    saved_target_sequences += sum(s.id in target_accessions for s in sequences_to_save)
print('After filtering:')
print('Discarded %i contaminants' % len(contaminant_seqids))
print('Discarded %i clusters without target sequences' % no_fungi_clusters)
print('Discarded %i predominantly fungal clusters' % predominantly_fungal_clusters)
# print('Discarded %i clusters with > 20 fungal sequences' % large_fungal_clusters)
print('Saved %i out of %i clusters with %i sequences total, %i target sequences' % (saved_clusters, len(cluster_fasta_files), saved_sequences, saved_target_sequences))

In [ ]:
filtered_cluster_files = os.listdir(filtered_cluster_directory)
cluster_sizes = []
for fstfl in filtered_cluster_files:
    S = SeqIO.parse(filtered_cluster_directory + '/' + fstfl, 'fasta')
    cluster_sizes.append(len(list(S)))
print(fname + ':')
print('Average cluster size:', sum(cluster_sizes)/len(cluster_sizes))
print('Largest cluster sizes:', sorted(cluster_sizes)[-10:])
print('Smallest cluster sizes:', sorted(cluster_sizes)[:10])